In [ ]:
import tensorflow as tf
from tensorflow import keras
from scipy.signal import convolve
import numpy as np
import matplotlib.pyplot as plt

### Prepare the inputs

In [ ]:
n_samples = 20
input_shape = (1, n_samples, 1)
tf.random.set_seed(5061983)
x = tf.random.uniform(input_shape)

### Build the model

In [ ]:
n_kernels = [4, 8]
kernel_size = [5, 5]
input_layer = keras.Input(shape=(n_samples,1), name='input')
layers = [input_layer]
kernel_init = 'glorot_uniform'
# kernel_init = 'ones'
for i,(n,sz) in enumerate(zip(n_kernels, kernel_size)):
    L = keras.layers.Conv1D(filters=n, kernel_size=sz, strides=1,
                            name=f'conv_{i+1}', padding='valid',
                            dilation_rate=1, groups=1, kernel_initializer=kernel_init,
                            use_bias=False)(layers[-1])
    layers.append(L)
model = keras.Model(inputs=layers[:1], outputs=layers[1:])
model.summary()

### Compute the output of the model

In [ ]:
y = model(x)

#### Get the weights, i.e., the kernels used in the convolutions

In [ ]:
conv_layers = model.layers[1:]
weights = [layer.weights[0] for layer in conv_layers]
weights_shapes = [w.shape for w in weights]
for shp,lyr in zip(weights_shapes, conv_layers):
    print(f'The weights of layer `{lyr.name}` have shape', shp)

### Figure out how Conv1D performs the computation

The first layer has `n_kernels[0]` 1D kernels of length `kernel_size[0]`. The input is a 1D time series of `n_samples` samples. The output will be a matrix with `n_samples - kernel_size[0] + 1` rows and `n_kernels[0]` columns. The i-th column is the convolution of the i-th kernel with the time series.

In [ ]:
y1 = np.array([convolve(np.squeeze(x.numpy()),
                        np.squeeze(weights[0][-1::-1, 0, i].numpy()),
                        mode='valid', method='direct') for i in range(n_kernels[0])]).T[np.newaxis,:,:]

In [ ]:
assert np.all(y1 == y[0].numpy())

The second layer has `n_kernels[1]` 2D kernels of size (`kernel_size[1]`,`n_kernels[0]`). The input is a 2D matrix with `n_samples - kernel_size[0] + 1` rows and `n_kernels[0]` columns. The output will be a matrix with `n_samples - kernel_size[0] - kernel_size[1] + 2` rows and `n_kernels[1]` columns. The i-th column is the __sum__ of the convolutions of __each column__ of the i-th 2D kernel with __each column__ of the 2D input matrix.

In [ ]:
y2 = np.array([np.array([convolve(np.squeeze(y[0][0, :, j].numpy()),
                         np.squeeze(weights[1][-1::-1, j, i].numpy()),
                         mode='valid', method='direct') for j in range(weights[1].shape[1])]).sum(axis=0) \
               for i in range(n_kernels[i])]).T[np.newaxis,:,:]

In [ ]:
assert np.max(np.abs(y2 - y[1].numpy())) < 1e-6